In [1]:
import re
import numpy as np
import pandas as pd
import json 
from batoomer.twitter_nodes.search_engine import GoogleSearchEngine
from googleapiclient.errors import HttpError
from tqdm.notebook import tqdm
import time

---
Load Google Credentials and initialize GoogleSearchEngine

In [2]:
keys = []
with open('../../../google_credentials.json', 'r') as f:
    keys = json.load(f)

se = GoogleSearchEngine(google_api_key=keys['api_key'], search_engine_id=keys['twitter_seID'])

In [3]:
data = pd.read_csv('Google-Search-poi-hania-test.csv', usecols=['poi'])
data.replace(np.nan, '', inplace=True)
data.head(5)

,poi
0,αποκορωνου εκκλησία παναγίας & αγ. δημητρίου
1,σφακιων βρυση
2,κανδανου- σελινου ολοκαυτωμα καντανου 1-3 ιουνιου
3,κανδανου- σελινου καρναβαλι παλαιοχωρας
4,αποκορωνου παρθεναγωγείο


In [4]:
queries = list(data['poi'])
results = pd.DataFrame()

for query in tqdm(queries):
    try:
        se.search(query=query, platform='facebook')
        result = se.get_results()
        results= results.append(result)
    except HttpError as err:
        if err.resp.status == 429:
            print('Api Limit Hit!')
            time.sleep(60 * 60 * 24)
            se.search(query=query, platform='facebook')
            result = se.get_results()
            results= results.append(result)

  0%|          | 0/100 [00:00<?, ?it/s]

Api Limit Hit!


In [6]:
results = (results
           .replace(np.nan, '')
           .rename(columns={'Query':'poi'})
           .reset_index()
           .drop('index', axis=1))
results.head(5)

,poi,Result 1,Result 2,Result 3,Result 4,Result 5,Result 6,Result 7,Result 8,Result 9,Result 10
0,αποκορωνου εκκλησία παναγίας & αγ. δημητρίου,https://www.facebook.com/enoriakounoupidiana/,https://www.facebook.com/enoritisstavrou/,https://www.facebook.com/pages/category/Church...,https://www.facebook.com/pages/category/Christ...,https://www.facebook.com/AgiosIoannisChanion/,,,,,
1,σφακιων βρυση,https://www.facebook.com/easyparkingsfakia/,https://www.facebook.com/pages/category/Religi...,https://www.facebook.com/pages/category/Local-...,https://www.facebook.com/BonsaiFlowersVrises/,,,,,,
2,κανδανου- σελινου ολοκαυτωμα καντανου 1-3 ιουνιου,,,,,,,,,,
3,κανδανου- σελινου καρναβαλι παλαιοχωρας,https://www.facebook.com/choreftikiomadaselino...,https://www.facebook.com/YRTAKINA/,https://www.facebook.com/hotelsofPaleochora/,https://www.facebook.com/BlueHorizonPAL/,https://www.facebook.com/bakeryanatoli/,,,,,
4,αποκορωνου παρθεναγωγείο,,,,,,,,,,


In [7]:
results.to_csv('Google-Search-poi-hania-test-facebook.csv', index=False)